In [70]:
"""
#################################
# Examine county data set

"""
import csv as csv
import numpy as np
import pandas as pd
import os
import sys
import time
import pickle
import datetime
import re
from pprint import pprint as pp

sys.path.append('/Users/geoffryder/opt/anaconda3/lib/python3.8/site-packages')  # so I can install new packages from here!

#################################
# Common functions
#pysqldf = lambda q: sqldf(q, globals())

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [71]:
# Wikipedia data: needs regex parsing to recover.  Contains MSA names, and counties within MSAs
data_folder = '/Users/geoffryder/Documents/557_expenses/personal/LWV/dev_lwv_tools/001_lwv_spreadsheet_sources'
# msa_path = os.path.join(data_folder, '2022JUNE_raw_data.txt')
# msa_path = os.path.join(data_folder, '2023MARCountyWide_20231017_160137_MarniRubin.txt')
# msa_path = os.path.join(data_folder, '2024Nov_CountyWide_20240529_102205_MarniRubin.txt')
msa_path = os.path.join(data_folder, 'david_precinct_outreach_data_voter_id.csv')

In [72]:
# msa_path = os.path.join(data_folder, '2024Nov_CountyWide_20240529_102205_MarniRubin.txt')
#df_raw = pd.read_csv(msa_path, sep='\t', encoding = "ISO-8859-1")
#print(df_raw.shape)

In [73]:
df_raw = pd.read_csv(msa_path, sep=',', encoding = "ISO-8859-1")
print(df_raw.shape)
#df_raw.iloc[0]

(18074, 53)


/var/folders/02/s6zj49qs4w765nx9988hlclr0000gq/T/ipykernel_63204/4071970400.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(msa_path, sep=',', encoding = "ISO-8859-1")


In [74]:
# The first dataset, focused on Belle Haven
path6 = os.path.join(data_folder, 'analytics_march_2024_email_list_6_pct_option.xlsx')
df6 = pd.read_excel(path6)
print(df6.shape)
df6['lwv_source'] = ['6_pct_option']*len(df6)
df6['VoterID6'] = df6['VoterID']
df6.head()

(3060, 8)


,VoterID,FirstName,LastName,MailAddress1,MailAddress2,MailCity,MailState,MailZip,lwv_source,VoterID6
0,97697763,Mary,Fagnani,617 5th Ave,Redwood City CA 94063,Redwood City,CA,94063,6_pct_option,97697763
1,64003528,Arlene,Rohrer,525 9th Ave,Menlo Park CA 94025,Menlo Park,CA,94025,6_pct_option,64003528
2,66003740,Joan,Schmitz,8 Athlone Ct,Menlo Park CA 94025,Menlo Park,CA,94025,6_pct_option,66003740
3,68000831,Doris,Wong,784 15th Ave,Menlo Park CA 94025,Menlo Park,CA,94025,6_pct_option,68000831
4,72000913,Pauline,Robinson,3675 Fair Oaks Ave,Menlo Park CA 94025,Menlo Park,CA,94025,6_pct_option,72000913


In [75]:
# The second dataset, focused on EPA
path27 = os.path.join(data_folder, 'analytics_march_2024_email_list_27_pct_option.xlsx')
df27 = pd.read_excel(path27)
print(df27.shape)
df27['lwv_source'] = ['27_pct_option']*len(df27)
df27['VoterID27'] = df27['VoterID']
df27.head()

(3117, 8)


,VoterID,FirstName,LastName,MailAddress1,MailAddress2,MailCity,MailState,MailZip,lwv_source,VoterID27
0,58001159,Anna,Davis,1324 Windermere Ave,Menlo Park CA 94025,Menlo Park,CA,94025,27_pct_option,58001159
1,58001651,Essie,Gant,1227 Madera Ave,Menlo Park CA 94025,Menlo Park,CA,94025,27_pct_option,58001651
2,97697763,Mary,Fagnani,617 5th Ave,Redwood City CA 94063,Redwood City,CA,94063,27_pct_option,97697763
3,60003033,Catherine,Daniels,1320 Henderson Ave,Menlo Park CA 94025,Menlo Park,CA,94025,27_pct_option,60003033
4,64003528,Arlene,Rohrer,525 9th Ave,Menlo Park CA 94025,Menlo Park,CA,94025,27_pct_option,64003528


In [76]:
df6_merge = df6[['VoterID6','lwv_source']].merge(df_raw, how='left', left_on='VoterID6', right_on='VoterID').drop(columns='VoterID6')
df6_merge[['VoterID','last_election_voted_date']].query("VoterID.notnull()").nunique()

VoterID                     2852
last_election_voted_date      29
dtype: int64

In [77]:
df27_merge = df27[['VoterID27','lwv_source']].merge(df_raw, how='left', left_on='VoterID27', right_on='VoterID').drop(columns='VoterID27')
df27_merge[['VoterID','last_election_voted_date']].query("VoterID.notnull()").nunique()

VoterID                     2898
last_election_voted_date      35
dtype: int64

In [78]:
df27_merge.head()

,lwv_source,Unnamed: 0,VoterID,Gender,Ethnicity,PartyName,Language,ResidenceCity,ResidenceState,ResidenceZipCode,StreetName,StreetSuffix,MailAddress1,MailAddress2,MailAddress3,MailAddress4,MailCity,MailState,MailZip,StatusCode,StatusReason,StateVoterStatus,VBMProgramStatus,PrecinctID,PrecinctName,ElectionAbbr_1,ElectionAbbr_2,ElectionAbbr_3,ElectionAbbr_4,ElectionAbbr_5,ElectionDesc_1,ElectionDesc_2,ElectionDesc_3,ElectionDesc_4,ElectionDesc_5,ElectionDate_1,ElectionDate_2,ElectionDate_3,ElectionDate_4,ElectionDate_5,ElectionType_1,ElectionType_2,ElectionType_3,ElectionType_4,ElectionType_5,original_registration_date,last_update_date,date_diff_reg2update_day,date_diff_update2now_day,date_diff_reg2update_yr,date_diff_update2now_yr,last_election_voted_date,date_diff_lastelection2now_day,date_diff_lastelection2now_yr
0,27_pct_option,272.0,58001159.0,F,NaN,Democratic,English,Menlo Park,CA,94025.0,Windermere,Ave,1324 Windermere Ave,Menlo Park CA 94025,NaN,NaN,Menlo Park,CA,94025,A,Original Registration,A,PermVBM,44039.0,Menlo Park 039,2024Mar,Nov2022,2022Jun,2020Nov,2020Mar,Presidential Primary Election,Statewide General Election,Statewide Direct Primary Election,Presidential General Election,Presidential Primary Election,03/05/2024,11/08/2022,06/07/2022,11/03/2020,03/03/2020,Primary,General,Primary,General,Primary,1958-08-13,2000-03-30,15205.0,8853.0,42.0,24.0,2024-03-05,112.0,0.0
1,27_pct_option,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27_pct_option,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27_pct_option,565.0,60003033.0,F,NaN,Democratic,English,Menlo Park,CA,94025.0,Henderson,Ave,1320 Henderson Ave,Menlo Park CA 94025,NaN,NaN,Menlo Park,CA,94025,A,Original Registration,A,PermVBM,44039.0,Menlo Park 039,2024Mar,Nov2022,2022Jun,2020Nov,2020Mar,Presidential Primary Election,Statewide General Election,Statewide Direct Primary Election,Presidential General Election,Presidential Primary Election,03/05/2024,11/08/2022,06/07/2022,11/03/2020,03/03/2020,Primary,General,Primary,General,Primary,1960-08-26,1999-10-13,14292.0,9022.0,39.0,25.0,2024-03-05,112.0,0.0
4,27_pct_option,892.0,64003528.0,F,NaN,Republican,English,Menlo Park,CA,94025.0,9th,Ave,525 9th Ave,Menlo Park CA 94025,NaN,NaN,Menlo Park,CA,94025,A,Adminstrative Action - Active,A,PermVBM,47508.0,Oaks 508,2024Mar,Nov2022,2022Jun,2020Nov,2020Mar,Presidential Primary Election,Statewide General Election,Statewide Direct Primary Election,Presidential General Election,Presidential Primary Election,03/05/2024,11/08/2022,06/07/2022,11/03/2020,03/03/2020,Primary,General,Primary,General,Primary,1964-09-08,2023-02-23,21352.0,488.0,58.0,1.0,2024-03-05,112.0,0.0


In [79]:
int_precincts = [47506, 40001, 40002, 40003, 40004, 40005, 40006, 40007, 40008, 40009, 40010, 40011, 40014, 40015, 40016, 40017, 40018, 44003, 44004, 44039, 47503, 47504, 47506, 47508, 47512, 47501, 47502, 47505, 47506, 47508, 47509, 47510, 47511, 47516]

df_baseline = df_raw.query(f"(PrecinctID in {int_precincts})")
print(df_baseline.shape)

(18074, 53)


In [80]:
def find_voting_rate(df_in):
    voted = -1
    all = -1
    voting_rate = -1
    try:
        #voted_in = df_in.query("LastElectionVoted == '2024-03-05 00:00:00'")['VoterID'].nunique()
        voted_in = df_in.query("last_election_voted_date == '2024-03-05'")['VoterID'].nunique()
        all_in = df_in['VoterID'].nunique()
        voting_rate_in = voted_in / all_in
    except Exception as e:
        print("Voting rate calculation failed : ", e)
    return voted_in, all_in, voting_rate_in
    

In [81]:
voted_6prec, all_6prec, voting_rate_6prec = find_voting_rate(df6_merge)
print(f"Flyer recipients who voted: {voted_6prec}, all flyer recipients: {all_6prec}, ratio voting: {round(voting_rate_6prec,3)}")
# df6_merge.LastElectionVoted.value_counts()  # 863

Flyer recipients who voted: 852, all flyer recipients: 2852, ratio voting: 0.299


In [82]:
voted_27prec, all_27prec, voting_rate_27prec = find_voting_rate(df27_merge)
print(f"Flyer recipients who voted: {voted_27prec}, all flyer recipients: {all_27prec}, ratio voting: {round(voting_rate_27prec,3)}")
# df6_merge.LastElectionVoted.value_counts()  # 769

Flyer recipients who voted: 759, all flyer recipients: 2898, ratio voting: 0.262


In [83]:
voted_baseline, all_baseline, voting_rate_baseline = find_voting_rate(df_baseline)
print(f"Area residents who voted: {voted_baseline}, all area residents: {all_baseline}, ratio voting: {round(voting_rate_baseline,3)}")
#df_baseline.LastElectionVoted.value_counts()  # 4152

Area residents who voted: 4152, all area residents: 18074, ratio voting: 0.23


In [84]:
# Here's one way to estimate the impact:  cost per additional voter over the average

In [85]:
added_6prec = voted_6prec - (voting_rate_baseline * all_6prec)
cost_per_voter_6prec = round(3000.0 / added_6prec, 3)
print(f"Campaign efficiency, 6 precinct list: ${round(cost_per_voter_6prec,2)} each for {round(added_6prec,0)} voters")

Campaign efficiency, 6 precinct list: $15.24 each for 197.0 voters


In [86]:
added_27prec = voted_27prec - (voting_rate_baseline * all_27prec)
cost_per_voter_27prec = round(3000.0 / added_27prec, 3)
print(f"Campaign efficiency, 27 precinct list: ${round(cost_per_voter_27prec,2)} each for {round(added_27prec,0)} voters")

Campaign efficiency, 27 precinct list: $32.17 each for 93.0 voters
